In [1]:
import os
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
import numpy as np


from ipywidgets import Layout, interact, IntSlider, FloatSlider
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")  # Report only TF errors by default

import DL_Sequence

num_classes = 5
res = 32

mode = "static"

dataset = DL_Sequence.iSCAT_DataGenerator(batch_size=4, epoch_size=1024, res=res, frames=32, thread_count=10,
                    PSF_path="../PSF_subpx_fl32.npy", exD=5000, devD=4000, exPT_cnt=500, devPT_cnt=499, exIntensity=1.0, devIntensity=0.3, target_frame=15,
                    num_classes=num_classes, verbose = 2, noise_func = None, mode = mode)

data = dataset.samples
targets = dataset.particles_in_sight_cnt
poss = dataset.particle_positions
pt_cnts = dataset.pt_cnts
dataset.destroy()


Sample width:  480
Run
Particle generation time: 0.081s
Sparse Subpixel Convolution initialized
Preparation of memory took 1623 ms

OMP max threads: 64
OMP setting number of threads to 10
Conv generation time: 0.122s
Target generation time: 0.007s
Whole generation time: 0.219s
Computation of convolutions took 110 ms
Run
Particle generation time: 0.085s
Min particle in sight cnt: 0
Max particle in sight cnt: 4
Avg particle in sight cnt: 1.953125
Std particle in sight cnt: 1.5096531834746019
class: 0     |1     |2     |3     |4     |
count: 241   |216   |170   |144   |253   |

OMP max threads: 10
OMP setting number of threads to 10
Conv generation time: 0.128s
Target generation time: 0.006s
Whole generation time: 0.227s
Computation of convolutions took 117 ms
Die


In [2]:
import tensorflow as tf

model_name = "models/model_dense_5000_simple_" + mode
model = tf.keras.models.load_model(model_name + ".h5")
predictions = tf.argmax(model.predict(data), axis = 1)
predictions = np.array(predictions)

print(data.shape, targets.shape)

print(np.max(data), np.min(data))
print(np.mean(targets), np.std(targets))

print("Accuracy: ", np.mean(predictions == targets))

32/32 [==============================] - 1s 2ms/step
(1024, 32, 32, 32) (1024,)
1.0 0.0
1.953125 1.5096531834746019
Accuracy:  0.4306640625


In [3]:
sample_inds = np.insert(np.cumsum(pt_cnts), 0, 0)

sample_size_px = 480
FOV_edge = sample_size_px / 2 - res / 2

def plot_step_fast(step, sample_cur):    
    imgs = data[sample_cur]
    
    begin = sample_inds[sample_cur]
    end = begin + pt_cnts[sample_cur]
    
    poss_step = step % poss.shape[0]
    
    x = poss[poss_step, begin:end, 1]
    y = poss[poss_step, begin:end, 0]
    
    plt.subplot(1, 2, 1)
    plt.axis([-FOV_edge, sample_size_px-FOV_edge, -FOV_edge, sample_size_px-FOV_edge])
    plt.gca().add_patch(Rectangle((0, 0), res, res, edgecolor = 'red', fill=False))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.scatter(x,y)
    
    plt.subplot(1, 2, 2)
    plt.imshow(imgs[step, :, :])
    plt.axis([-0.5, res-0.5, -0.5, res-0.5])
    plt.scatter(x,y, c="red")
    plt.title("data")
    plt.show()
    
    print("gold:", targets[sample_cur])
    print("pred:", int(predictions[sample_cur]))


    
interact(plot_step_fast, step=IntSlider(min=0, max=data.shape[1] - 1), sample_cur=IntSlider(min=0, max=data.shape[0] - 1));

interactive(children=(IntSlider(value=0, description='step', max=31), IntSlider(value=0, description='sample_c…